In [2]:
import pandas as pd
# Importing the statsmodels function for proportion testing
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

# Load mock data
df = pd.DataFrame({
    'user_id': range(2000),
    'group': np.random.choice(['A', 'B'], 2000, p=[0.5, 0.5]),
    'converted': np.random.choice([0, 1], 2000, p=[0.95, 0.05])
})


In [3]:
df.loc[df['group'] == 'B', 'converted'] = np.random.choice([0, 1], len(df[df['group'] == 'B']), p=[0.93, 0.07])

# Calculate sample size (N), conversions (X), and conversion rate (CR)
summary = df.groupby('group')['converted'].agg(
    N='count',
    X='sum',
    CR='mean'
).reset_index()

# Extract key metrics
N_A, X_A, CR_A = summary.loc[summary['group'] == 'A', ['N', 'X', 'CR']].values[0]
N_B, X_B, CR_B = summary.loc[summary['group'] == 'B', ['N', 'X', 'CR']].values[0]

print("--- A/B Test Summary ---")
print(summary)

--- A/B Test Summary ---
  group     N   X        CR
0     A  1016  53  0.052165
1     B   984  56  0.056911


In [4]:
# The z-test requires the number of successes (X) and number of trials (N) for both groups.
count = np.array([X_B, X_A]) # Successes (conversions) for B, then A
nobs = np.array([N_B, N_A])  # Observations (sample size) for B, then A

# Perform the Z-test (using a two-sided test, value=0 for difference)
# statsmodels returns the Z-score and the P-value directly.
Z_score, p_value = proportions_ztest(count, nobs, alternative='two-sided')

print(f"\n--- Statistical Results (using statsmodels) ---")
print(f"Z-Score: {Z_score:.4f}")
print(f"P-Value: {p_value:.4f}")


--- Statistical Results (using statsmodels) ---
Z-Score: 0.4674
P-Value: 0.6402


In [5]:
alpha = 0.05 # Significance level defined in Section 2

print("\n--- Conclusion ---")
if p_value < alpha:
    print(f"Result: REJECT the Null Hypothesis (H0) because P-Value ({p_value:.4f}) < Alpha ({alpha}).")
    print("Interpretation: The difference in conversion rates is statistically significant.")
    print("Recommendation: Adopt Campaign B, as it performs significantly better than A.")
else:
    print(f"Result: FAIL TO REJECT the Null Hypothesis (H0) because P-Value ({p_value:.4f}) >= Alpha ({alpha}).")
    print("Interpretation: The observed difference is not statistically significant and is likely due to chance.")
    print("Recommendation: Do NOT adopt Campaign B based solely on this metric, or continue testing to achieve significance.")




--- Conclusion ---
Result: FAIL TO REJECT the Null Hypothesis (H0) because P-Value (0.6402) >= Alpha (0.05).
Interpretation: The observed difference is not statistically significant and is likely due to chance.
Recommendation: Do NOT adopt Campaign B based solely on this metric, or continue testing to achieve significance.
